In [1]:
import json
import re

import requests as rq
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd
from shapely.geometry import Polygon

from ee_secret_codes import *

In [2]:
class Landsat(object):

    def __init__(self, username, password):

        # Define dataset search params
        self.datasets = {"LANDSAT_TM_C1": "Thematic Mapper",
                         "LANDSAT_ETM_C1": "Enhanced Thematic Matter",
                         "LANDSAT_8_C1": "Collection 1"}

        # Define endpoints
        self.endpoint = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/"
        self.ee_login_endpoint = self.endpoint + "login?"
        self.ee_search_endpoint = self.endpoint + "search"
        self.ee_status_endpoint = self.endpoint + "status"
    
        # Login parameters
        self.username = username
        self.password = password

        # Set up auths dict
        self.auths_dict = {'username': self.username,
                           'password': self.password,
                           'catalogID': "EE"}

        self.api_key = self.getApiKey()

    def print_datasets(self):
        print("Search Datasets\n---------------")
        for k, v in self.datasets.items():
            print(k, ":", v)

    def create_request(self, request_dict):
        return {'jsonRequest': json.dumps(request_dict)}

    def getApiKey(self):
        # Format auths
        login_auths = self.create_request(self.auths_dict)

        # Post auths and return json
        login = rq.post(self.ee_login_endpoint, data=login_auths).json()

        # Return API key
        return login['data']
    
    def getStatus(self):
        status_request = rq.get(self.ee_status_endpoint)
        return status_request.json()

    def results_overview(self, response_json):
        if response_json['errorCode'] == None:

            total_hits = response_json['data']['totalHits']

            print(f"{total_hits} total scenes(s), only 10 available")

            """Return datasets list was here"""

        else:
            print(f"Failed with an error code of {results_json['errorCode']}")

    def search(self, dataset_name, startDate, endDate, latitude=None, longitude=None):
        # Create empty dict
        search_payload = {}

        # Add dataset name
        search_payload["datasetName"] = dataset_name

        # Add temporal filter
        search_payload["temporalFilter"] = {"startDate": startDate,
                                            "endDate": endDate}

        # If latitude and longitude
        if latitude and longitude:
            search_payload['spatialFilter'] = {"filterType": "mbr",
                                               "lowerLeft": {"latitude": latitude,
                                                             "longitude": longitude},
                                               "upperRight": {"latitude": latitude,
                                                              "longitude": longitude}
                                               }

        # Add API key
        search_payload["apiKey"] = self.api_key

        # Get results as json
        search_resp = rq.get(self.ee_search_endpoint,
                             params=self.create_request(search_payload))
        
        # Response object to json
        search_resp_json = search_resp.json()
        
        # Pass json to results_overview function to get simplified info
        self.results_overview(search_resp_json)
        
        # Return gdf created from return_gdf()
        # Commented out for testing
        return self.get_gdf(search_resp_json)
        
        return search_resp_json

    def get_gdf(self, search_resp_json):
                  
        # normalize json to create df
        df = json_normalize(search_resp_json['data']['results'])

        # Create list to hold Polygon object from each row
        rows_list = []

        # Loop through each scene item in json
        for scene in search_resp_json['data']['results']:

            # DataFrame holds Polygons as a nested list, need to
            # change format to [(x1, y1), (x2, y2)] for shapely
            # Start with "packed list" in df
            packed_list = scene.get('spatialFootprint').get('coordinates')[0]

            # Change each item to a tuple and create a simple list
            unpacked_list = [tuple(sublist) for sublist in packed_list]

            # Create Polygon object for each lit of tuples
            rows_list.append(Polygon(unpacked_list))

        # Create new column and pass it the list
        # of Polygon objects, one for each row
        df['geometry'] = rows_list

        # Create gdf from dataframe, pass in CRS
        gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
                  
        # Drop columns
        gdf.drop(columns=['bulkOrdered', 'ordered', 'summary', 'spatialFootprint.coordinates'],
                inplace=True)
        
        #pd.set_option('display.max_colwidth', -1)
        
        return gdf

# Instantiate class
S = Landsat(EE_USERNAME, EE_PASSWORD)

In [27]:
j = S.search("LANDSAT_8_C1", "2020-01-01", "2020-04-19", 40, -105)
j

14 total scenes(s), only 10 available


,acquisitionDate,startTime,endTime,sceneBounds,browseUrl,dataAccessUrl,downloadUrl,entityId,displayId,cloudCover,metadataUrl,fgdcMetadataUrl,modifiedDate,orderUrl,spatialFootprint.type,geometry
0,2020-01-01,2020-01-01,2020-01-01,"-105.57121,39.26348,-102.84417,41.38644",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80330322020001LGN00,LC08_L1TP_033032_20200101_20200113_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-01-13,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-105.57121 39.66940, -103.41774 39.2..."
1,2020-01-08,2020-01-08,2020-01-08,"-107.11546,39.26397,-104.38995,41.38672",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80340322020008LGN00,LC08_L1TP_034032_20200108_20200114_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-01-13,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-107.11546 39.66960, -104.96356 39.2..."
2,2020-01-17,2020-01-17,2020-01-17,"-105.57468,39.26393,-102.84758,41.38682",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80330322020017LGN00,LC08_L1TP_033032_20200117_20200128_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-01-27,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-105.57468 39.66980, -103.42103 39.2..."
3,2020-01-24,2020-01-24,2020-01-24,"-107.11801,39.26389,-104.39259,41.38658",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80340322020024LGN00,LC08_L1TP_034032_20200124_20200128_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-01-28,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-107.11801 39.66942, -104.96608 39.2..."
4,2020-02-02,2020-02-02,2020-02-02,"-105.57312,39.26369,-102.84625,41.38649",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80330322020033LGN00,LC08_L1TP_033032_20200202_20200211_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-02-11,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-105.57312 39.66943, -103.41961 39.2..."
5,2020-02-09,2020-02-09,2020-02-09,"-107.12127,39.26381,-104.39619,41.38639",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80340322020040LGN00,LC08_L1GT_034032_20200209_20200211_01_T2,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-02-12,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-107.12127 39.66918, -104.96951 39.2..."
6,2020-02-18,2020-02-18,2020-02-18,"-105.58413,39.26377,-102.85759,41.3865",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplorer.usgs.gov/download/extern...,LC80330322020049LGN00,LC08_L1TP_033032_20200218_20200225_01_T1,None,https://earthexplorer.usgs.gov/metadata/xml/12...,https://earthexplorer.usgs.gov/fgdc/12864/LC80...,2020-02-25,https://earthexplorer.usgs.gov/order/process?d...,Polygon,"POLYGON ((-105.58413 39.66937, -103.43084 39.2..."
7,2020-02-25,2020-02-25,2020-02-25,"-107.13389,39.26392,-104.40929,41.38644",https://ims.cr.usgs.gov/browse/landsat_8_c1/20...,https://earthexplorer.usgs.gov/order/process?d...,https://earthexplo

In [26]:
def landsat_name_parser(scenes_list):
    
    # Regular expression to parse landsat filename
    exp = r"L(\w)0(\d)_(.{4})_(\d{3})(\d{3})_(\d{8})_(\d{8})_(\d{2})_(.{2})"
    
    # List to hold scene data
    rows_list = []
    
    # Main loop, for each element in list
    for scene_name in scenes_list:
    
        mo = re.search(exp, scene_name)

        if mo is not None:

            # Lookup sensor value
            sensor_types = {"C": "OLI/TIRS Combined",
                            "O": "OLI Only",
                            "T": "TIRS Only or TM",
                            "E": "ETM+",
                            "M": "MSS"}

            try:
                sensor = sensor_types[mo.group(1)]
            except:
                sensor = mo.group(1)

            # Lookup collection category
            collection_categories = {"RT": "real-time",
                                     "T1": "Tier 1",
                                     "T2": 'Tier 2'}

            try:
                collection_category = collection_categories[mo.group(9)]
            except:
                collection_category = mo.group(9)

            # Create dict of variables from scene
            d = {}

            d['scene_name'] = scene_name
            d['sensor'] = sensor
            d['satellite'] = f"Landsat {mo.group(2)}"
            d['processing_correction_level'] = mo.group(3)
            d['wrs_path'] = mo.group(4)
            d['wrs_row'] = mo.group(5)
            d['collected'] = (mo.group(6))
            d['processed'] = mo.group(7)
            d['collection_number'] = mo.group(8)
            d['collection_category'] = collection_category

            rows_list.append(d)

        else:
            pass
    
    return pd.DataFrame(rows_list)

scenes_list = ['LC08_L1TP_033032_20200406_20200410_01_T1',
               'LC08_L1TP_024099_20200501_20200410_01_RT']

df = landsat_name_parser(scenes_list)

df

,scene_name,sensor,satellite,processing_correction_level,wrs_path,wrs_row,collected,processed,collection_number,collection_category
0,LC08_L1TP_033032_20200406_20200410_01_T1,OLI/TIRS Combined,Landsat 8,L1TP,033,032,20200406,20200410,01,Tier 1
1,LC08_L1TP_024099_20200501_20200410_01_RT,OLI/TIRS Combined,Landsat 8,L1TP,024,099,20200501,20200410,01,real-time


In [13]:
basket_data

{'jsonRequest': '{"apiKey": "8a1044305de84166bbbd4c1a98651c56"}'}

In [14]:
ee_basket = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/itembasket"
basket_payload = {"apiKey": S.api_key}
basket_data = S.create_request(basket_payload)
basket = rq.get(ee_basket, params=basket_data)
basket.json()

{'errorCode': None,
 'error': '',
 'data': {'bulkDownloadItemBasket': [], 'orderItemBasket': []},
 'api_version': '1.4.0',
 'access_level': 'approved',
 'catalog_id': 'EE',
 'executionTime': 0.07392311096191406}

In [28]:
# Get order products
order_prod_url = "https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/getorderproducts"
order_prod_dict = {"datasetName": "LANDSAT_8_C1",
                   "apiKey": S.api_key,
                   "entityIds": ["LC80330322020001LGN00"]}
order_prod_params = S.create_request(order_prod_dict)
order_prods = rq.get(order_prod_url, params=order_prod_params).json()
order_prods['data'][0]

{'errorCode': None,
 'error': '',
 'data': [{'entityId': 'LC80330322020001LGN00',
   'availableProducts': [{'options': ['None'],
     'outputMedias': ['DWNLD'],
     'price': 0,
     'productCode': 'W015',
     'productName': 'L8 OLI/TIRS L1 WMS ON-DEMAND'}],
   'product': None}],
 'api_version': '1.4.0',
 'access_level': 'approved',
 'catalog_id': 'EE',
 'executionTime': 0.34593796730041504}

In [19]:
# Add item to basket
ee_order_url = 'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/updateorderscene'
order_payload = {"apiKey": S.api_key,
                 "datasetName": "LANDSAT_8_C1",
                 "entityId": "LC08_L1TP_033032_20200406_20200410_01_T1",
                 "productCode": "T273",
                 "option": "None",
                 "outputMedia": "DWNLD"}

order_params = S.create_request(order_payload)

order = rq.get(ee_order_url, params=order_params).json()
order

{'errorCode': 'UNKNOWN',
 'error': 'Invalid scene ID',
 'data': None,
 'api_version': '1.4.0',
 'access_level': 'approved',
 'catalog_id': 'EE',
 'executionTime': 0.2963900566101074}